In [4]:
# !pip install ib_insync
# !pip install schedule
# !pip install ibapi

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ibapi: filename=ibapi-9.81.1.post1-py3-none-any.whl size=67965 sha256=f682f9bf7e9b1c8b7b1f1edbfdb0fac349aedd14dea3f8ba80f0b61c0e376782
  Stored in directory: c:\users\desir\appdata\local\pip\cache\wheels\07\e7\d4\b94d0968474f56f8a2c3eafe3debcc340f15ef1ba362eb38a8
Successfully built ibapi


In [10]:
from ib_insync import *
import requests
import pandas as pd
import schedule
import time
import datetime;
from bs4 import BeautifulSoup
import warnings
from ibapi.client import EClient
#from ibapi.order_cancel import OrderCancel
warnings.filterwarnings("ignore", category=RuntimeWarning)

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)

# https://github.com/PacktPublishing/Python-for-Algorithmic-Trading-Cookbook/issues/7
# https://www.interactivebrokers.com/campus/ibkr-api-page/order-types/#market-on-open-order 
class IBClient(EClient):

    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)
    
    def cancel_all_orders(self):
        self.reqGlobalCancel()

    def cancel_order_by_id(self, order_id):
        order_cancel = OrderCancel()
        self.cancelOrder(orderId=order_id, orderCancel=order_cancel)

    def update_order(self, contract, order, order_id):
        self.cancel_order_by_id(order_id)
        return self.send_order(contract=contract, order=order)



In [12]:
ib = IB()
ib.connect(host='127.0.0.1', port=7496, clientId=0)
# https://www.elitetrader.com/et/threads/cant-connect-to-api-interactive-brokers.361205/
# needed to change socket port to 7496 and uncheck the Read-Only API box

<IB connected to 127.0.0.1:7496 clientId=0>

In [16]:
pending_stop_loss_orders = pd.DataFrame() # define and create one time
pos = ib.positions()
pos

[Position(account='U12848716', contract=Stock(conId=700941285, symbol='NUVOQ', exchange='PINK', currency='USD', localSymbol='NUVOQ', tradingClass='NUVOQ'), position=275.0, avgCost=0.24124875),
 Position(account='U12848716', contract=Stock(conId=656170396, symbol='CDT', exchange='NASDAQ', currency='USD', localSymbol='CDT', tradingClass='NMS'), position=4.0, avgCost=0.129425),
 Position(account='U12848716', contract=Stock(conId=447584486, symbol='ZOM', exchange='AMEX', currency='USD', localSymbol='ZOM', tradingClass='ZOM'), position=1.0, avgCost=0.14),
 Position(account='U12848716', contract=Stock(conId=586262891, symbol='SYRS', exchange='NASDAQ', currency='USD', localSymbol='SYRS', tradingClass='NMS'), position=1.0, avgCost=0.2193)]

In [ ]:
# util.logToConsole()

def order_status(trade):
 # okay wtf DOES trade brimg with it exactly? 
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        fill
        #for index, row in pending_stop_loss_orders.iterrows():
        #    if(row['TK'] == fill.contract.symbol and row['QT'] == fill.execution.shares):
                
        contract = Stock('TSLA', 'SMART', 'USD')
        stopOrder = StopOrder('BUY', fill.execution.shares, 10.10)
        stopOrderTrade = ib.placeOrder(contract, stopOrder)
        stopOrderTrade
                
                #pending_stop_loss_orders.drop(index, inplace=True)   # no longer useful to keep (does this work?)
                
                
# test orders with day high volatility stocks
sym = 'GLXG'
test_symbols = [sym, sym]



for test in test_symbols:
    
    contract = Stock(test, 'SMART', 'USD')

    ib.qualifyContracts(contract)
    data = ib.reqMktData(contract)
    ib.sleep(2) # data.marketPrice() doesn't work without ib.sleep (few seconds)
    current_price = data.marketPrice()
    
    limitOrder = LimitOrder('BUY', 1, round(current_price * 1.015, 2))    
    limitOrderTrade = ib.placeOrder(contract, limitOrder)
    # limitOrderTrade contains the order and everything related to it, such as order status, fills and a log.
    # It will be live updated with every status change or fill of the order.
        
    # CREATE A DATAFRAME THAT TRACKS STOP LIMIT ORDERS MADE ALONG WITH THEIR STOP LOSS TARGETS
    new_row = pd.DataFrame({"TK": [limitOrderTrade.contract.symbol], "QT": [limitOrderTrade.order.totalQuantity], 
                            "OID": [limitOrderTrade.order.orderId], "SLT": [round(current_price * 1.25, 2)]})
    pending_stop_loss_orders = pd.concat([pending_stop_loss_orders, new_row], ignore_index=True)
    
    limitOrderTrade.fillEvent += order_status

In [ ]:
trads = ib.trades()
trads

In [ ]:
trads = ib.trades()
trads

In [ ]:
trads[0].contract.symbol

In [ ]:
trads[0].order.orderId

In [ ]:
trads[0].orderStatus.status

In [18]:
ords = ib.orders()
ords

[Order(orderId=-3, permId=1800011630, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=1.25, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U12848716', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True, usePriceMgmtAlgo=True),
 Order(orderId=-4, permId=2057958630, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=1.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U12848716', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True, usePriceMgmtAlgo=True)]

In [28]:
ib.cancelOrder(ords[1])
# this seems to work now... was it always like this or was adding the EClient class the difference??

Trade(contract=Stock(conId=618449783, symbol='NYMT', right='?', exchange='SMART', currency='USD', localSymbol='NYMT', tradingClass='NMS'), order=Order(orderId=-4, permId=2057958630, action='BUY', totalQuantity=1.0, orderType='LMT', lmtPrice=1.0, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U12848716', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True, usePriceMgmtAlgo=True), orderStatus=OrderStatus(orderId=-4, status='PendingCancel', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=2057958630, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2025, 1, 15, 12, 32, 10, 171455, tzinfo=datetime.timezone.utc), status='Submitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2025, 1, 15, 12, 44, 4, 3046, tzinfo=datetime.timezone.utc), stat

In [ ]:
fills = ib.fills()
fills

In [ ]:
contract = Stock('ZOM', 'SMART', 'USD')

ib.qualifyContracts(contract)
data = ib.reqMktData(contract)
ib.sleep(2)
# data.last / data.marketPrice()
    
limitOrder = LimitOrder('BUY', 1, (round(data.marketPrice() * 1.015, 2)))
limitOrderTrade = ib.placeOrder(contract, limitOrder)

In [ ]:
limitOrderTrade

In [ ]:
limitOrderTrade.contract.symbol

In [ ]:
limitOrderTrade.order.orderId

In [ ]:
limitOrderTrade.order.totalQuantity